In [1]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights="imagenet",
    input_shape=(224, 224, 3),
    include_top=False)

In [2]:
base_model.trainable = False

In [3]:
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1, activation = 'softmax')(x)
model = keras.Model(inputs,outputs)

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________


In [5]:
model.compile(loss = "categorical_crossentropy" , metrics = ["categorical_accuracy"])

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        samplewise_center=True,  
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip=True,  
        vertical_flip=True
)

In [7]:
train_it = datagen.flow_from_directory("./fruits/train", 
                                       target_size=(224,224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")

valid_it = datagen.flow_from_directory("./fruits/valid", 
                                      target_size=(224,224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=12)

Epoch 1/12
36/36 [==============================] - 44s 708ms/step - loss: 0.6033 - categorical_accuracy: 0.8096 - val_loss: 0.2873 - val_categorical_accuracy: 0.9271
Epoch 2/12
36/36 [==============================] - 14s 386ms/step - loss: 0.2205 - categorical_accuracy: 0.9467 - val_loss: 0.1753 - val_categorical_accuracy: 0.9635
Epoch 3/12
36/36 [==============================] - 16s 424ms/step - loss: 0.1484 - categorical_accuracy: 0.9611 - val_loss: 0.1958 - val_categorical_accuracy: 0.9362
Epoch 4/12
36/36 [==============================] - 15s 396ms/step - loss: 0.1139 - categorical_accuracy: 0.9746 - val_loss: 0.1066 - val_categorical_accuracy: 0.9726
Epoch 5/12
36/36 [==============================] - 16s 443ms/step - loss: 0.0938 - categorical_accuracy: 0.9755 - val_loss: 0.1264 - val_categorical_accuracy: 0.9726
Epoch 6/12
36/36 [==============================] - 15s 396ms/step - loss: 0.0737 - categorical_accuracy: 0.9805 - val_loss: 0.1148 - val_categorical_accuracy: 0.969

In [ ]:
base_model.trainable = True

model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),
              loss = "categorical_crossentropy" , metrics = ["categorical_accuracy"])

In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=12)

In [ ]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

In [ ]:
model.save("fruit_model.h5")